<a href="https://colab.research.google.com/github/rishi255/softlab-a3/blob/main/softlab_data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU omdb
%pip install -qU torch
!git clone https://github.com/huggingface/transformers

from bs4 import BeautifulSoup
import json
import omdb
import requests
import re
import traceback
import torch

print("All installed and imported!")

Cloning into 'transformers'...
remote: Enumerating objects: 377, done.
remote: Counting objects: 100% (377/377), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 69566 (delta 197), reused 296 (delta 152), pack-reused 69189
Receiving objects: 100% (69566/69566), 53.05 MiB | 22.09 MiB/s, done.
Resolving deltas: 100% (49217/49217), done.
All installed and imported!


# Data Scraping

In [21]:
# get screenplay of all movies returned in the search
def scrape(name_with_dash_list):
    screenplays = []
    print(name_with_dash_list)
    
    # for movie_id, movie_title in movies_list:
    for movie_name_with_dash in name_with_dash_list: 
        print("For movie", movie_name_with_dash)
        # name_with_dash = "-".join(movie_title.split())
        screenplay_url = f"https://imsdb.com/scripts/{movie_name_with_dash}.html"
        print("Attempting to get the screenplay from url", screenplay_url)
        # XPATH = /html/body/table[2]/tbody/tr/td[3]/table/tbody/tr/td/pre/pre
        response = requests.request(method="get", url=screenplay_url)
        soup = BeautifulSoup(response.text, "html.parser")
        try:
            screenplay_text = soup.find('td', class_="scrtext").pre.pre
            print("Here", type(screenplay_text), str(screenplay_text)[:300])
            try:
                screenplay_text = screenplay_text.text
            except:
                screenplay_text = ""
        except:
            print(f"Movie {movie_name_with_dash} not found!")
            traceback.print_exc()
            # return None
        else:
            screenplays.append(screenplay_text)
            print(f"*** {movie_name_with_dash} done!")    

    with open("screenplays.txt", "w+") as f:
        f.writelines(screenplays)

scrape(["Jurassic-Park", "Jurassic-Park-III", "Jurassic-Park-The-Lost-World"])

['Jurassic-Park', 'Jurassic-Park-III', 'Jurassic-Park-The-Lost-World']
For movie Jurassic-Park
Attempting to get the screenplay from url https://imsdb.com/scripts/Jurassic-Park.html
Here <class 'bs4.element.Tag'> <pre>



<b>                                JURASSIC PARK
</b>


                                 First Draft
                                     by
<b>                              MICHAEL CRICHTON
</b>


                                 Re-write by
<b>                             M
*** Jurassic-Park done!
For movie Jurassic-Park-III
Attempting to get the screenplay from url https://imsdb.com/scripts/Jurassic-Park-III.html
Here <class 'NoneType'> None
*** Jurassic-Park-III done!
For movie Jurassic-Park-The-Lost-World
Attempting to get the screenplay from url https://imsdb.com/scripts/Jurassic-Park-The-Lost-World.html
Here <class 'bs4.element.Tag'> <pre>
<b>			THE LOST WORLD
</b><b>			 JURASSIC PARK
</b>
			Screenplay by
			  David Koepp
<b>	
</b>		     based on the novel by


# Data Cleanup

In [22]:
# !cat JP1.txt JP2.txt JP3.txt > merged.txt
# !wc -l JP1.txt
# !wc -l JP2.txt
# !wc -l JP3.txt

!wc -l screenplays.txt
# !wc -c merged.txt

13769 screenplays.txt


# Actual Training and Predicting stuff



In [ ]:
%pip install transformers/
%pip install -r transformers/examples/language-modeling/requirements.txt

In [ ]:
!ls
!pwd
!python transformers/examples/language-modeling/run_clm.py \
    --output_dir=output \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --do_train \
    --train_file=/content/merged.txt \
    --overwrite_output_dir\
    --block_size=200\
    --per_device_train_batch_size=1\
    --save_steps 5000\
    --num_train_epochs=2

In [ ]:
!python transformers/examples/text-generation/run_generation.py \
    --model_type gpt2 \
    --model_name_or_path output \
    --length 1000 \
    --prompt "The T-Rex let out a loud roar." \
    > test.txt

04/05/2021 22:16:30 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
04/05/2021 22:16:45 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=1000, model_name_or_path='output', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.9, padding_text='', prefix='', prompt='The velociraptor leaned in for a kiss.', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
2021-04-05 22:17:04.969620: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
